In [ ]:
!rm -rf /content/CosPlace

In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/gmberton/CosPlace.git
%cd ./CosPlace/
!gdown https://drive.google.com/u/0/uc?id=14U3jsoNEWC-QsINoVCWZaHFUGE20fIgZ&export=download
!gdown https://drive.google.com/u/0/uc?id=1WYqU2pNGjooON05fVcp2F55MqMiIckJB&export=download
# !gdown https://drive.google.com/u/0/uc?id=1tQqEyt3go3vMh4fj_LZrRcahoTbzzH-y&export=download
# !unzip SF_XS_Train_&_Val.zip
!gdown https://drive.google.com/u/0/uc?id=15QB3VNKj93027UAQWv7pzFQO1JDCdZj2&export=download
!unzip tokyo_xs.zip
!pip install faiss_cpu
!pip install timm
!pip install efficientnet-pytorch

Streaming output truncated to the last 5000 lines.
  inflating: tokyo_xs/test/database/@0382337.61@3946371.54@54@S@035.65418@0139.70017@7iGzzRJ20rkcq3Y7eVyZ2A@07@@@@@@@.jpg  
  inflating: tokyo_xs/test/database/@0382337.61@3946371.54@54@S@035.65418@0139.70017@7iGzzRJ20rkcq3Y7eVyZ2A@08@@@@@@@.jpg  
  inflating: tokyo_xs/test/database/@0382337.61@3946371.54@54@S@035.65418@0139.70017@7iGzzRJ20rkcq3Y7eVyZ2A@09@@@@@@@.jpg  
  inflating: tokyo_xs/test/database/@0382337.91@3946962.38@54@S@035.65950@0139.70009@2Xcp44iJEgEd16nSr5XGYA@00@@@@@@@.jpg  
  inflating: tokyo_xs/test/database/@0382337.91@3946962.38@54@S@035.65950@0139.70009@2Xcp44iJEgEd16nSr5XGYA@03@@@@@@@.jpg  
  inflating: tokyo_xs/test/database/@0382337.91@3946962.38@54@S@035.65950@0139.70009@2Xcp44iJEgEd16nSr5XGYA@04@@@@@@@.jpg  
  inflating: tokyo_xs/test/database/@0382337.91@3946962.38@54@S@035.65950@0139.70009@2Xcp44iJEgEd16nSr5XGYA@08@@@@@@@.jpg  
  inflating: tokyo_xs/test/database/@0382337.91@3946962.38@54@S@035.65950@0139.70

In [ ]:
%cd /content/CosPlace/

/content/CosPlace


In [ ]:
%%writefile train.py


import sys
import torch
import logging
import numpy as np
from tqdm import tqdm
import multiprocessing
from datetime import datetime
import torchvision.transforms as T

import test
import util
import parser
import commons
import cosface_loss
import augmentations
from cosplace_model import cosplace_network
from datasets.test_dataset import TestDataset
from datasets.train_dataset import TrainDataset

torch.backends.cudnn.benchmark = True  # Provides a speedup

args = parser.parse_arguments()
start_time = datetime.now()
args.output_folder = f"logs/{args.save_dir}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"
commons.make_deterministic(args.seed)
commons.setup_logging(args.output_folder, console="debug")
logging.info(" ".join(sys.argv))
logging.info(f"Arguments: {args}")
logging.info(f"The outputs are being saved in {args.output_folder}")

#### Model
model = cosplace_network.GeoLocalizationNet(args.backbone, args.fc_output_dim)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
    logging.debug(f"Loading model from {args.resume_model}")
    model_state_dict = torch.load(args.resume_model)
    model.load_state_dict(model_state_dict)

model = model.to(args.device).train()

#### Optimizer
criterion = torch.nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

#### Datasets
groups = [TrainDataset(args, args.train_set_folder, M=args.M, alpha=args.alpha, N=args.N, L=args.L,
                       current_group=n, min_images_per_class=args.min_images_per_class) for n in range(args.groups_num)]
# Each group has its own classifier, which depends on the number of classes in the group
classifiers = [cosface_loss.MarginCosineProduct(args.fc_output_dim, len(group)) for group in groups]
classifiers_optimizers = [torch.optim.Adam(classifier.parameters(), lr=args.classifiers_lr) for classifier in classifiers]

logging.info(f"Using {len(groups)} groups")
logging.info(f"The {len(groups)} groups have respectively the following number of classes {[len(g) for g in groups]}")
logging.info(f"The {len(groups)} groups have respectively the following number of images {[g.get_images_num() for g in groups]}")

val_ds = TestDataset(args.val_set_folder, positive_dist_threshold=args.positive_dist_threshold)
test_ds = TestDataset(args.test_set_folder, queries_folder="queries",
                      positive_dist_threshold=args.positive_dist_threshold)
logging.info(f"Validation set: {val_ds}")
logging.info(f"Test set: {test_ds}")

#### Resume
if args.resume_train:
    model, model_optimizer, classifiers, classifiers_optimizers, best_val_recall1, start_epoch_num = \
        util.resume_train(args, args.output_folder, model, model_optimizer, classifiers, classifiers_optimizers)
    model = model.to(args.device)
    epoch_num = start_epoch_num - 1
    logging.info(f"Resuming from epoch {start_epoch_num} with best R@1 {best_val_recall1:.1f} from checkpoint {args.resume_train}")
else:
    best_val_recall1 = start_epoch_num = 0

#### Train / evaluation loop
logging.info("Start training ...")
logging.info(f"There are {len(groups[0])} classes for the first group, " +
             f"each epoch has {args.iterations_per_epoch} iterations " +
             f"with batch_size {args.batch_size}, therefore the model sees each class (on average) " +
             f"{args.iterations_per_epoch * args.batch_size / len(groups[0]):.1f} times per epoch")


if args.augmentation_device == "cuda":
    gpu_augmentation = T.Compose([
            augmentations.DeviceAgnosticColorJitter(brightness=args.brightness,
                                                    contrast=args.contrast,
                                                    saturation=args.saturation,
                                                    hue=args.hue),
            augmentations.DeviceAgnosticRandomResizedCrop([512, 512],
                                                          scale=[1-args.random_resized_crop, 1]),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

if args.use_amp16:
    scaler = torch.cuda.amp.GradScaler()

for epoch_num in range(start_epoch_num, args.epochs_num):

    #### Train
    epoch_start_time = datetime.now()
    # Select classifier and dataloader according to epoch
    current_group_num = epoch_num % args.groups_num
    classifiers[current_group_num] = classifiers[current_group_num].to(args.device)
    util.move_to_device(classifiers_optimizers[current_group_num], args.device)

    dataloader = commons.InfiniteDataLoader(groups[current_group_num], num_workers=args.num_workers,
                                            batch_size=args.batch_size, shuffle=True,
                                            pin_memory=(args.device == "cuda"), drop_last=True)

    dataloader_iterator = iter(dataloader)
    model = model.train()

    epoch_losses = np.zeros((0, 1), dtype=np.float32)
    for iteration in tqdm(range(args.iterations_per_epoch), ncols=100):
        images, targets, _ = next(dataloader_iterator)
        images, targets = images.to(args.device), targets.to(args.device)

        if args.augmentation_device == "cuda":
            images = gpu_augmentation(images)

        model_optimizer.zero_grad()
        classifiers_optimizers[current_group_num].zero_grad()

        if not args.use_amp16:
            descriptors = model(images)
            output = classifiers[current_group_num](descriptors, targets)
            loss = criterion(output, targets)
            loss.backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            model_optimizer.step()
            classifiers_optimizers[current_group_num].step()
        else:  # Use AMP 16
            with torch.cuda.amp.autocast():
                descriptors = model(images)
                output = classifiers[current_group_num](descriptors, targets)
                loss = criterion(output, targets)
            scaler.scale(loss).backward()
            epoch_losses = np.append(epoch_losses, loss.item())
            del loss, output, images
            scaler.step(model_optimizer)
            scaler.step(classifiers_optimizers[current_group_num])
            scaler.update()

    classifiers[current_group_num] = classifiers[current_group_num].cpu()
    util.move_to_device(classifiers_optimizers[current_group_num], "cpu")

    logging.debug(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, "
                  f"loss = {epoch_losses.mean():.4f}")

    #### Evaluation
    recalls, recalls_str = test.test(args, val_ds, model)
    logging.info(f"Epoch {epoch_num:02d} in {str(datetime.now() - epoch_start_time)[:-7]}, {val_ds}: {recalls_str[:20]}")
    is_best = recalls[0] > best_val_recall1
    best_val_recall1 = max(recalls[0], best_val_recall1)
    # Save checkpoint, which contains all training parameters
    util.save_checkpoint({
        "epoch_num": epoch_num + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": model_optimizer.state_dict(),
        "classifiers_state_dict": [c.state_dict() for c in classifiers],
        "optimizers_state_dict": [c.state_dict() for c in classifiers_optimizers],
        "best_val_recall1": best_val_recall1
    }, is_best, args.output_folder)


logging.info(f"Trained for {epoch_num+1:02d} epochs, in total in {str(datetime.now() - start_time)[:-7]}")

#### Test best model on test set v1
best_model_state_dict = torch.load(f"{args.output_folder}/best_model.pth")
model.load_state_dict(best_model_state_dict)

logging.info(f"Now testing on the test set: {test_ds}")
recalls, recalls_str = test.test(args, test_ds, model, args.num_preds_to_save)
logging.info(f"{test_ds}: {recalls_str}")

logging.info("Experiment finished (without any errors)")



Overwriting train.py


In [ ]:
%cd /content/CosPlace/datasets/

/content/CosPlace/datasets


In [ ]:
%%writefile train_dataset.py


import os
import torch
import random
import logging
import numpy as np
from glob import glob
from PIL import Image
from PIL import ImageFile
import torchvision.transforms as T
from collections import defaultdict

ImageFile.LOAD_TRUNCATED_IMAGES = True


def open_image(path):
    return Image.open(path).convert("RGB")


class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, args, dataset_folder, M=10, alpha=30, N=5, L=2,
                 current_group=0, min_images_per_class=10):
        """
        Parameters (please check our paper for a clearer explanation of the parameters).
        ----------
        args : args for data augmentation
        dataset_folder : str, the path of the folder with the train images.
        M : int, the length of the side of each cell in meters.
        alpha : int, size of each class in degrees.
        N : int, distance (M-wise) between two classes of the same group.
        L : int, distance (alpha-wise) between two classes of the same group.
        current_group : int, which one of the groups to consider.
        min_images_per_class : int, minimum number of image in a class.
        """
        super().__init__()
        self.M = M
        self.alpha = alpha
        self.N = N
        self.L = L
        self.current_group = current_group
        self.dataset_folder = dataset_folder
        self.augmentation_device = args.augmentation_device

        # dataset_name should be either "processed", "small" or "raw", if you're using SF-XL
        dataset_name = os.path.basename(args.dataset_folder)
        filename = f"cache/{dataset_name}_M{M}_N{N}_mipc{min_images_per_class}.torch"
        if not os.path.exists(filename):
            os.makedirs("cache", exist_ok=True)
            logging.info(f"Cached dataset {filename} does not exist, I'll create it now.")
            self.initialize(dataset_folder, M, N, alpha, L, min_images_per_class, filename)
        elif current_group == 0:
            logging.info(f"Using cached dataset {filename}")

        classes_per_group, self.images_per_class = torch.load(filename)
        if current_group >= len(classes_per_group):
            raise ValueError(f"With this configuration there are only {len(classes_per_group)} " +
                             f"groups, therefore I can't create the {current_group}th group. " +
                             "You should reduce the number of groups by setting for example " +
                             f"'--groups_num {current_group}'")
        self.classes_ids = classes_per_group[current_group]

        if self.augmentation_device == "cpu":
            self.transform = T.Compose([
                    T.ColorJitter(brightness=args.brightness,
                                  contrast=args.contrast,
                                  saturation=args.saturation,
                                  hue=args.hue),
                    T.RandomResizedCrop([512, 512], scale=[1-args.random_resized_crop, 1]),
                    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])

    def __getitem__(self, class_num):
        # This function takes as input the class_num instead of the index of
        # the image. This way each class is equally represented during training.

        class_id = self.classes_ids[class_num]
        # Pick a random image among those in this class.
        image_path = random.choice(self.images_per_class[class_id])

        try:
            pil_image = open_image(image_path)
        except Exception as e:
            logging.info(f"ERROR image {image_path} couldn't be opened, it might be corrupted.")
            raise e

        tensor_image = T.functional.to_tensor(pil_image)

        if self.augmentation_device == "cpu":
            tensor_image = self.transform(tensor_image)

        return tensor_image, class_num, image_path

    def get_images_num(self):
        """Return the number of images within this group."""
        return sum([len(self.images_per_class[c]) for c in self.classes_ids])

    def __len__(self):
        """Return the number of classes within this group."""
        return len(self.classes_ids)

    @staticmethod
    def initialize(dataset_folder, M, N, alpha, L, min_images_per_class, filename):
        logging.debug(f"Searching training images in {dataset_folder}")

        if not os.path.exists(dataset_folder):
            raise FileNotFoundError(f"Folder {dataset_folder} does not exist")

        images_paths = sorted(glob(f"{dataset_folder}/**/*.jpg", recursive=True))
        logging.debug(f"Found {len(images_paths)} images")

        logging.debug("For each image, get its UTM east, UTM north and heading from its path")
        images_metadatas = [p.split("@") for p in images_paths]
        # field 1 is UTM east, field 2 is UTM north, field 9 is heading
        utmeast_utmnorth_heading = [(m[1], m[2], m[9]) for m in images_metadatas]
        utmeast_utmnorth_heading = np.array(utmeast_utmnorth_heading).astype(np.float)

        logging.debug("For each image, get class and group to which it belongs")
        class_id__group_id = [TrainDataset.get__class_id__group_id(*m, M, alpha, N, L)
                              for m in utmeast_utmnorth_heading]

        logging.debug("Group together images belonging to the same class")
        images_per_class = defaultdict(list)
        for image_path, (class_id, _) in zip(images_paths, class_id__group_id):
            images_per_class[class_id].append(image_path)

        # Images_per_class is a dict where the key is class_id, and the value
        # is a list with the paths of images within that class.
        images_per_class = {k: v for k, v in images_per_class.items() if len(v) >= min_images_per_class}

        logging.debug("Group together classes belonging to the same group")
        # Classes_per_group is a dict where the key is group_id, and the value
        # is a list with the class_ids belonging to that group.
        classes_per_group = defaultdict(set)
        for class_id, group_id in class_id__group_id:
            if class_id not in images_per_class:
                continue  # Skip classes with too few images
            classes_per_group[group_id].add(class_id)

        # Convert classes_per_group to a list of lists.
        # Each sublist represents the classes within a group.
        classes_per_group = [list(c) for c in classes_per_group.values()]

        torch.save((classes_per_group, images_per_class), filename)

    @staticmethod
    def get__class_id__group_id(utm_east, utm_north, heading, M, alpha, N, L):
        """Return class_id and group_id for a given point.
            The class_id is a triplet (tuple) of UTM_east, UTM_north and
            heading (e.g. (396520, 4983800,120)).
            The group_id represents the group to which the class belongs
            (e.g. (0, 1, 0)), and it is between (0, 0, 0) and (N, N, L).
        """
        rounded_utm_east = int(utm_east // M * M)  # Rounded to nearest lower multiple of M
        rounded_utm_north = int(utm_north // M * M)
        rounded_heading = int(heading // alpha * alpha)

        class_id = (rounded_utm_east, rounded_utm_north, rounded_heading)
        # group_id goes from (0, 0, 0) to (N, N, L)
        group_id = (rounded_utm_east % (M * N) // M,
                    rounded_utm_north % (M * N) // M,
                    rounded_heading % (alpha * L) // alpha)
        return class_id, group_id



Overwriting train_dataset.py


In [ ]:
%cd /content/CosPlace/

/content/CosPlace


In [ ]:
!unzip SF_XS_Only_Train.zip

Streaming output truncated to the last 5000 lines.
  inflating: SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/val/queries/@0548258.48@4179554.69@10@S@037.76204@-122.45211@GGArRnpN7_AJXIx-El-9XQ@@90@@@@200711@@.jpg  
  inflating: SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/val/queries/@0548258.51@4180452.30@10@S@037.77013@-122.45205@HhjqWdMQqByOIcdqhG7-eg@@0@@@@201402@@.jpg  
  inflating: SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/val/queries/@0548258.51@4180452.30@10@S@037.77013@-122.45205@HhjqWdMQqByOIcdqhG7-eg@@120@@@@201402@@.jpg  
  inflating: SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/val/queries/@0548258.51@4180452.30@10@S@037.77013@-122.45205@HhjqWdMQqByOIcdqhG7-eg@@180@@@@201402@@.jpg  
  inflating: SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/val/queries/@0548258.51@4180452.30@10@S@037.77013@-122.45205@HhjqWdMQqByOIcdqhG7-eg@@210@@@@201402@@.jpg  
  infla

In [ ]:
!python3 train.py --dataset_folder ./SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/ --groups=1 --iterations_per_epoch 1000 --epochs_num 10

2023-06-07 15:59:50   train.py --dataset_folder ./SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/ --groups=1 --iterations_per_epoch 1000 --epochs_num 10
2023-06-07 15:59:50   Arguments: Namespace(M=10, alpha=30, N=5, L=2, groups_num=1, min_images_per_class=10, backbone='ResNet18', fc_output_dim=512, use_amp16=False, augmentation_device='cuda', batch_size=32, epochs_num=10, iterations_per_epoch=1000, lr=1e-05, classifiers_lr=0.01, brightness=0.7, contrast=0.7, hue=0.5, saturation=0.7, random_resized_crop=0.5, infer_batch_size=16, positive_dist_threshold=25, resume_train=None, resume_model=None, device='cuda', seed=0, num_workers=8, num_preds_to_save=0, save_only_wrong_preds=False, dataset_folder='./SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/', save_dir='default', train_set_folder='./SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/sf_xs/train', val_set_folder='./SF_XS_Only_Train_Dataset_Half_Transformed_To_Night_Time_Images/s

In [ ]:
!python3 eval.py --dataset_folder /content/CosPlace/tokyo_xs/ --backbone ResNet18 --fc_output_dim 512 --resume_model /content/best_model.pth

2023-06-15 15:26:54   eval.py --dataset_folder /content/CosPlace/tokyo_xs/ --backbone ResNet18 --fc_output_dim 512 --resume_model /content/best_model.pth
2023-06-15 15:26:54   Arguments: Namespace(M=10, alpha=30, N=5, L=2, groups_num=8, min_images_per_class=10, backbone='ResNet18', fc_output_dim=512, use_amp16=False, augmentation_device='cuda', batch_size=32, epochs_num=50, iterations_per_epoch=10000, lr=1e-05, classifiers_lr=0.01, brightness=0.7, contrast=0.7, hue=0.5, saturation=0.7, random_resized_crop=0.5, infer_batch_size=16, positive_dist_threshold=25, resume_train=None, resume_model='/content/best_model.pth', device='cuda', seed=0, num_workers=8, num_preds_to_save=0, save_only_wrong_preds=False, dataset_folder='/content/CosPlace/tokyo_xs/', save_dir='default', test_set_folder='/content/CosPlace/tokyo_xs/test', output_folder='logs/default/2023-06-15_15-26-54')
2023-06-15 15:26:54   The outputs are being saved in logs/default/2023-06-15_15-26-54
2023-06-15 15:26:54   There are 1 G

In [ ]:
!python3 eval.py -h

usage: eval.py
       [-h]
       [--M M]
       [--alpha ALPHA]
       [--N N]
       [--L L]
       [--groups_num GROUPS_NUM]
       [--min_images_per_class MIN_IMAGES_PER_CLASS]
       [--backbone {VGG16,ResNet18,ResNet50,ResNet101,ResNet152}]
       [--fc_output_dim FC_OUTPUT_DIM]
       [--use_amp16]
       [--augmentation_device {cuda,cpu}]
       [--batch_size BATCH_SIZE]
       [--epochs_num EPOCHS_NUM]
       [--iterations_per_epoch ITERATIONS_PER_EPOCH]
       [--lr LR]
       [--classifiers_lr CLASSIFIERS_LR]
       [--brightness BRIGHTNESS]
       [--contrast CONTRAST]
       [--hue HUE]
       [--saturation SATURATION]
       [--random_resized_crop RANDOM_RESIZED_CROP]
       [--infer_batch_size INFER_BATCH_SIZE]
       [--positive_dist_threshold POSITIVE_DIST_THRESHOLD]
       [--resume_train RESUME_TRAIN]
       [--resume_model RESUME_MODEL]
       [--device {cuda,cpu}]
       [--seed SEED]
       [--num_workers NUM_WORKERS]
       [--num_preds_to_save NUM_PREDS_TO_SAVE]

In [ ]:
!rm -rf /content/CosPlace/sf_xs/test/queries
!rm -rf /content/CosPlace/sf_xs/test/database

In [ ]:
!unzip /content/CosPlace/queries.zip -d /content/CosPlace/sf_xs/test/

unzip:  cannot find or open /content/CosPlace/queries.zip, /content/CosPlace/queries.zip.zip or /content/CosPlace/queries.zip.ZIP.


In [ ]:
!python3 eval.py --dataset_folder '/content/CosPlace/sf_xs/' --backbone 'ResNet18' --fc_output_dim 512 --resume_model '/content/CosPlace/best_model.pth'

Traceback (most recent call last):
  File "/content/CosPlace/eval.py", line 16, in <module>
    args = parser.parse_arguments(is_training=False)
  File "/content/CosPlace/parser.py", line 75, in parse_arguments
    raise FileNotFoundError(f"Folder {args.dataset_folder} does not exist")
FileNotFoundError: Folder /content/CosPlace/sf_xs/ does not exist
